# Ensemble methods--Bagging & Boosting

  在上一篇[Full_Project_Process](https://nbviewer.jupyter.org/github/superjcd/ML-DL/blob/master/Full_Project_Process.ipynb#)里我们为了预测房价，训练了几种不同的模型,最终表现最好（依据cross_val_score）的模型是alpha值为0.1的lasso回归，其平均交叉验证误差为69052.4。这一回，我将使用一种machine learning领域非常强有力的训练方法（Essemble methods）来改进模型的预测效果。essemble models的基本思想就是与其训练一个好的模型，不如训练若干个表现一般的模型然后将它们的结果ensemble起来（对于回归问题，比如各个模型预测值的平均），实践证明essemble models通常具有比一般模型更好的预测效果，也是这几年kaggle比赛中表现最好的两种主流方法之一（另一种是deep learnng）。

## 1 Data preparing

In [1]:
#ML_tools 汇集了上一篇中所用过的若干类
from ML_tools import *

In [2]:
#采取和上次一样的数据分割方法
import pandas as pd
import numpy as np

House = pd.read_csv('Data/housing.csv')
House["income_cat"] = np.ceil(House["median_income"] / 1.5)
# Label those above 5 as 5
House["income_cat"].where(House["income_cat"] < 5, 5.0, inplace=True)

from sklearn.model_selection import StratifiedShuffleSplit

strait = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42) #strait split提供n对保留原有数据结构的train，test index
for train_index, test_index in strait.split(House, House["income_cat"]): #.split(X,y)
    strat_train_set = House.loc[train_index]
    strat_test_set = House.loc[test_index]

In [3]:
Train_x = strat_train_set.drop(['median_house_value'],axis=1)

In [4]:
Train_y = strat_train_set['median_house_value'].copy()

In [5]:
Train_x.shape,Train_y.shape #和之前是一模一样的

((16512, 10), (16512,))

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer

num_attribs = Train_x.columns[:-2]
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([   #list of tuples,the easy way is make_pipeline
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', Imputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)),
        ('cat_encoder', CategoricalEncoder(encoding="onehot-dense")),
    ])

full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

Train_x= full_pipeline.fit_transform(Train_x)

## 2 Bagging --a sample of decision tree

  模型因为会尽可能”记住“训练数据，所以往往在新数据上过拟合。为了减少由于variance带来的error(error = bias+variance)，我们可以在模型训练的时候对数据加入variance，这样模型就只能‘记住’部分而不是全部，这样就可以减少test error。对于回归问题，我们只要对各个由随机数据集获得的模型的预测值进行平均(分类问题使用投票)，就能得到最终的预测值。

In [7]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

bg_reg = BaggingRegressor(DecisionTreeRegressor(random_state=42),
                         n_estimators=100, #训练100个模型
                          max_samples=200,  #每次选取200个数据
                          bootstrap=True,  #使用bootsrap方法，即放回式sample，允许重复
                         n_jobs =-1,   #使用尽可能多的cpu
                          random_state=42)

In [8]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(bg_reg,Train_x,Train_y,cv=5,scoring='neg_mean_squared_error')
cvscore_bg_tree = np.sqrt(-score).mean()
cvscore_bg_tree #通过bagging改进的决策树模型要远远低于原有的误差,值得注意的是如果是BaggingCalssier的话，模型自带obb_score

60675.64829676476

## 3 Random Forest

 一般的bagging方法，像上面提到的这种，是对数据进行随机选取自己，然后进行训练。随机森林，在此基础上，对变量也进行了一次随机选取，这样
可以极大地提升模型在验证集上的表现。

In [9]:
from sklearn.ensemble import RandomForestRegressor

In [10]:
rf_reg = RandomForestRegressor(n_estimators=200,
                               n_jobs=-1, 
                               random_state=42)

In [11]:
score = cross_val_score(rf_reg,Train_x,Train_y,scoring='neg_mean_squared_error',cv=5)

In [12]:
cvscore_rf = np.sqrt(-score).mean()
cvscore_rf #我们选取的参数获得较好的预测效果，但是是不是能更近一步呢

50273.665635143254

In [13]:
#fine tune the model
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

rf_reg = RandomForestRegressor(random_state=42)

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=10),
    }#如果我们使用grid_search的话将会由200*10=2000种可能性

rnd_search = RandomizedSearchCV(rf_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)#多了n_iter参数

rnd_search.fit(Train_x,Train_y)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=42,
           verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1147abe10>, 'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1158d06a0>},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score=True, scoring='neg_mean_squared_error',
          verbose=0)

In [14]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)#可以看到最小的error为49147要远远小于前面的所有方法

49147.15241724505 {'max_features': 7, 'n_estimators': 180}
49054.217812866234 {'max_features': 8, 'n_estimators': 189}
49581.75837312377 {'max_features': 5, 'n_estimators': 103}
50776.736049370644 {'max_features': 3, 'n_estimators': 75}
49142.307571955236 {'max_features': 8, 'n_estimators': 117}
49621.00032993271 {'max_features': 4, 'n_estimators': 104}
49133.89163784365 {'max_features': 8, 'n_estimators': 131}
49649.95109531419 {'max_features': 6, 'n_estimators': 53}
52008.30195205342 {'max_features': 2, 'n_estimators': 88}
49330.20930219184 {'max_features': 6, 'n_estimators': 130}


In [15]:
best_rf_model=rnd_search.best_estimator_

In [16]:
#验证最好的模型在test set上的表现
Test_x = strat_test_set.drop(['median_house_value'],axis=1)
Test_y = strat_test_set['median_house_value']

Test_x = full_pipeline.fit_transform(Test_x)

In [17]:
best_rf_model.fit(Train_x,Train_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=8, max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=189, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [18]:
from sklearn.metrics import mean_absolute_error
rf_predictions = best_rf_model.predict(Test_x)

#mean_absolute_error
mean_absolute_error(Test_y,rf_predictions) #误差不到5万

49795.67310738895

In [19]:
#feature importance 
np.argmax(best_rf_model.feature_importances_)

7

In [20]:
House.columns[7] #meadian_income 做

'median_income'

## 4 Adaboost

In [21]:
from sklearn.ensemble import AdaBoostRegressor
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=1),n_estimators=200, learning_rate=0.5, random_state=42)

In [22]:
score = cross_val_score(ada_reg,Train_x,Train_y,cv=5,scoring='neg_mean_squared_error')

cvscore_ada_tree = np.sqrt(-score).mean()
cvscore_ada_tree

94855.57241601839

## 5 Gradiant Boosting
  Gradiant的基本原理是这样的，第一步，用简单的模型（比如线形模型和深度较浅的回归树模型）在Train_set上训练，然后对上一步的误差进行进一步的
 训练，然后重复这个过程。在预测的时候只要把各个模型的预测值加起来即可，如下下图所示（来源：Hands_on_machine_learning）：

 ![示意图](./fig/GradiantBoost.png)

In [23]:
from sklearn.ensemble import GradientBoostingRegressor

In [24]:
gbrt = GradientBoostingRegressor(learning_rate=0.1,
                                n_estimators=100,
                                max_depth=3,
                                min_samples_split=3)

In [25]:
gbrt.fit(Train_x,Train_y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=3,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [26]:
predictions_gbrt = gbrt.predict(Train_x)

In [27]:
train_error_gbrt = mean_absolute_error(Train_y,predictions_gbrt)
train_error_gbrt #train error is low

35102.22759200338

In [28]:
#cross val score
score = cross_val_score(gbrt,Train_x,Train_y,cv=5,scoring='neg_mean_squared_error')

In [29]:
cvscore_gbrt = np.sqrt(-score).mean()
cvscore_gbrt #略高于前面的随机森里

53276.77228589257

如何更进一步地提升模型？那么就需要改变我们的gradiantboostingregressor 的参数。 
方法有以下三种：   
1、Gridsearch cross validation   
2、Randomsearch cross validaton   
3、early-stop（可以给予不同的指标，比如mse是不是持续下降，当然也可以结合AIC，BIC的指标）  

In [30]:
#这里我们采用一种结合random search 和 grid search的方法
gbrt = GradientBoostingRegressor(random_state=42)

param_distribs = {
        'learning_rate':[0.1,0.3,0.5,1],
        'n_estimators':randint(low=40, high=300),
        'max_features': randint(low=1, high=10),
        'max_depth': randint(low=1, high=10),
        'min_samples_leaf':randint(low=1,high=10)
    }

rsc_gbtr=RandomizedSearchCV(gbrt,param_distributions=param_distribs,
                                n_iter=30, cv=5, scoring='neg_mean_squared_error', random_state=42)

In [31]:
rsc_gbtr.fit(Train_x,Train_y)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=42, subsample=1.0, verbose=0,
             warm_start=False),
          fit_params={}, iid=True, n_iter=30, n_jobs=1,
          param_distributions={'learning_rate': [0.1, 0.3, 0.5, 1], 'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1158fc5c0>, 'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1158fc400>, 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1158fc2e8>, 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x115907048>},
          pre_dispatch='2*n_jobs', rando

In [32]:
rscv = rsc_gbtr.cv_results_

for  mean_score,params in zip(rscv['mean_test_score'],rscv['params']):
    print(np.sqrt(-mean_score),params)

50896.58408943678 {'learning_rate': 0.5, 'max_depth': 4, 'max_features': 8, 'min_samples_leaf': 5, 'n_estimators': 142}
50288.3299513258 {'learning_rate': 0.3, 'max_depth': 3, 'max_features': 7, 'min_samples_leaf': 8, 'n_estimators': 139}
72770.56255296334 {'learning_rate': 1, 'max_depth': 8, 'max_features': 3, 'min_samples_leaf': 6, 'n_estimators': 297}
68654.39026826981 {'learning_rate': 1, 'max_depth': 6, 'max_features': 2, 'min_samples_leaf': 5, 'n_estimators': 200}
65918.08962251447 {'learning_rate': 1, 'max_depth': 6, 'max_features': 9, 'min_samples_leaf': 1, 'n_estimators': 98}
50666.7897326738 {'learning_rate': 0.5, 'max_depth': 3, 'max_features': 7, 'min_samples_leaf': 4, 'n_estimators': 170}
50995.20462696798 {'learning_rate': 0.1, 'max_depth': 3, 'max_features': 7, 'min_samples_leaf': 5, 'n_estimators': 206}
49540.402704992186 {'learning_rate': 0.3, 'max_depth': 4, 'max_features': 9, 'min_samples_leaf': 2, 'n_estimators': 92}
50357.649971547165 {'learning_rate': 0.3, 'max_de

In [33]:
prams_data = pd.DataFrame(list(rscv['params']))

In [34]:
prams_data['Error']= np.sqrt(-rscv['mean_test_score'])

In [35]:
prams_data.sort_values('Error') 

,learning_rate,max_depth,max_features,min_samples_leaf,n_estimators,Error
10,0.3,6,6,4,230,48645.930298
7,0.3,4,9,2,92,49540.402705
25,0.5,4,7,3,125,50094.764016
1,0.3,3,7,8,139,50288.329951
8,0.3,4,7,8,74,50357.649972
5,0.5,3,7,4,170,50666.789733
0,0.5,4,8,5,142,50896.584089
6,0.1,3,7,5,206,50995.204627
22,0.5,7,9,3,102,52104.638119
11,0.3,2,4,8,254,52540.433154


在前两组表现较好的模型参数的急促上，在进行grid search方法：

In [36]:
from sklearn.model_selection import GridSearchCV

params = [{'max_depth':[4,6],
         'max_features':[6,7,9],
         'min_samples_leaf':[2,3,4],
         'n_estimators':[100,200]}] #36种可能

gscv_gbrt = GridSearchCV(GradientBoostingRegressor(learning_rate=0.3,random_state=42),params,cv=5, 
                          scoring='neg_mean_squared_error')

In [37]:
gscv_gbrt.fit(Train_x,Train_y)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.3, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=42, subsample=1.0, verbose=0,
             warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'max_depth': [4, 6], 'max_features': [6, 7, 9], 'min_samples_leaf': [2, 3, 4], 'n_estimators': [100, 200]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [38]:
gs_cv = gscv_gbrt.cv_results_
for  mean_score,params in zip(gs_cv['mean_test_score'],gs_cv['params']):
    print(np.sqrt(-mean_score),params)


49968.36617931869 {'max_depth': 4, 'max_features': 6, 'min_samples_leaf': 2, 'n_estimators': 100}
48752.66224492643 {'max_depth': 4, 'max_features': 6, 'min_samples_leaf': 2, 'n_estimators': 200}
50025.44582826646 {'max_depth': 4, 'max_features': 6, 'min_samples_leaf': 3, 'n_estimators': 100}
49037.7090251143 {'max_depth': 4, 'max_features': 6, 'min_samples_leaf': 3, 'n_estimators': 200}
50208.23304943882 {'max_depth': 4, 'max_features': 6, 'min_samples_leaf': 4, 'n_estimators': 100}
48922.74446727734 {'max_depth': 4, 'max_features': 6, 'min_samples_leaf': 4, 'n_estimators': 200}
49402.42256475303 {'max_depth': 4, 'max_features': 7, 'min_samples_leaf': 2, 'n_estimators': 100}
48383.049078433374 {'max_depth': 4, 'max_features': 7, 'min_samples_leaf': 2, 'n_estimators': 200}
49337.00503184393 {'max_depth': 4, 'max_features': 7, 'min_samples_leaf': 3, 'n_estimators': 100}
48435.110273715014 {'max_depth': 4, 'max_features': 7, 'min_samples_leaf': 3, 'n_estimators': 200}
49633.64746498155 {

In [39]:
np.sqrt(-gscv_gbrt.best_score_),gscv_gbrt.best_params_ #可以看到当我们选择以下参数的时候，获得了不到48000的误差，
#是一个非常不错的表现了

(47939.63566240527,
 {'max_depth': 6,
  'max_features': 9,
  'min_samples_leaf': 4,
  'n_estimators': 200})

 接下来如果我还要继续改进模型的话，该怎么做？一种方法是，用我们训练的模型对test_set对数据进行验证，判断模型在那一个区间或者说data space的预测效果较好，根据预测效果不好的data space进行进一步的探索（是数据量的问题？需要增加新的feature？还是说需要用新的模型来拟合这块预测表现不好的区域？然后也可以继续结合gradian boost的方法，对残差进行拟合，然后emsenble来获得最好的预测效果。

## Summary

  Emsemble方法的基本思想就是训练多个可能表现一般的模型，然后通过合并模型的方式来提升模型的预测表现，常用的方法包括bagging和boosting。bagging是通过集合在不完整的数据集上的模型来减少对训练集的‘记忆’，实践证明这种方法往往可以减少由variance带来的误差，其中随机森林在bagging的基础上，加入了对fearture的少量随机选取，能够更好地改善模型表现。Boosting方法，包括adaboost和gradiantboost，前者通过提升误差较大的预测值在loss function中的比重来来增强对误差数据的预测表现。后者则是通过连续地对误差地迭代训练，并合并多个模型来提升预测表现。可以看到不管是随机森林还是gradiant boost的表现都要远远地好于一般的线性模型。